In [4]:
import keras
import numpy as np
import tensorflow as tf


import tensorflow as tf
from layers import Msg, LinEq2v2, LinEq2v0, InputLayer
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential




In [5]:
def PELICAN(
        msg_outputs,
        agg_outputs,
        scal_outputs,
        dense_output,
        dropout=0.0,
        activation=None):

    layers = [InputLayer()]

    assert(len(msg_outputs) == len(agg_outputs))
    for msg_out, agg_out in zip(msg_outputs, agg_outputs):
        msg = Msg(outputs=msg_out, activation=activation)
        drop = Dropout(dropout)
        agg = LinEq2v2(outputs=agg_out, activation=activation)
        layers += [msg, drop, agg]


    layers += [
        Dropout(dropout),
        LinEq2v0(outputs=scal_outputs, activation=activation),
        Flatten()
    ]

    layers += [
        Dense(units=num_units, activation=activation) for num_units in dense_output[:-1]
    ] + [Dense(units=dense_output[-1], activation='softmax')]

    return Sequential(layers=layers, name="PELICAN")

In [6]:
model = PELICAN(
    msg_outputs=(10, 10, 20),
    agg_outputs=(20, 20, 30),
    scal_outputs=21,
    dense_output=(100, 10),
    activation='leaky_relu',
    dropout=0.2,
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)



In [7]:
a = np.random.normal(size=(20, 3))


In [8]:
A = np.array(
[
        np.einsum('ij, kj->ik', elem, elem) for elem in [
        np.random.permutation(a) for _ in range(100)
    ]
])

A = np.expand_dims(A, -1)


In [9]:
print(A.shape)
y = model.predict(A)
yavg = np.average(y, axis=0)


(100, 20, 20, 1)



4/4 [==============================] - 3s 24ms/step


In [10]:
print(np.std(y-yavg))
print(np.mean(y-yavg))

0.0
0.0


In [11]:
print(y.shape)
print(y)


(100, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [12]:
model.summary()

Model: "PELICAN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer_1 (InputLayer)  (None, 20, 20, 1)         0         
                                                                 
 msg (Msg)                   (None, 20, 20, 10)        50        
                                                                 
 dropout (Dropout)           (None, 20, 20, 10)        0         
                                                                 
 lin_eq2v2 (LinEq2v2)        (None, 20, 20, 20)        3000      
                                                                 
 msg_1 (Msg)                 (None, 20, 20, 10)        240       
                                                                 
 dropout_1 (Dropout)         (None, 20, 20, 10)        0         
                                                                 
 lin_eq2v2_1 (LinEq2v2)      (None, 20, 20, 20)        3000